## Энергетический оракул
Ноутбук команды #12

Работа выполнена на основе модели LightGBM


### 1. Подготовка данных

In [1]:
import numpy as np
import pandas as pd


import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

import xgboost as xgb
from xgboost import XGBRegressor


import re

from tqdm import tqdm
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True )


from data_preprocess import DataTransformer

random_state = 12345
NUM_ITERATIONS = 5000

FEATURES = 'TEST'

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
# Функция для вычисления метрики mae по дням из почасовых массивов данных

def mae_day(y_true, y_pred):
    y_true_copy = pd.DataFrame(y_true).reset_index(drop=True)
    y_true_copy['day'] = y_true_copy.index // 24
    y_true_grouped = y_true_copy.groupby(by='day').sum()   
    y_pred_copy = pd.DataFrame(y_pred).reset_index(drop=True)
    y_pred_copy['day'] = y_pred_copy.index // 24
    y_pred_grouped = y_pred_copy.groupby(by='day').sum()
    
    return mean_absolute_error(y_true_grouped, y_pred_grouped)
# Функция для вычисления метрик по дням из почасовых массивов данных

def metrics_hour(y_true, y_pred):

    
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, mape, r2

#### 1.5 Чтение файлов с данными
Данные объединяются в один датасет

In [3]:
transformer = DataTransformer()

In [4]:
path = 'test/train_test_test_dataset.csv'

train_ds, test_begin, test_end = transformer.open_file(path)



i am here


In [5]:
test_begin, test_end

(Timestamp('2023-08-01 00:00:00'), Timestamp('2023-10-01 00:00:00'))

In [6]:
train_ds

,date,time,target,temp,temp_pred,weather_pred,weather_fact
0,2019-01-01,0,481.510,2.9,2.0,"пасм, ветер",ветер
1,2019-01-01,1,462.872,2.9,2.0,"пасм, ветер",ветер
2,2019-01-01,2,449.718,2.9,2.0,"пасм, ветер",ветер
3,2019-01-01,3,430.908,4.3,2.0,"пасм, ветер","ветер, пасм"
4,2019-01-01,4,415.163,4.3,2.0,"пасм, ветер","ветер, пасм"
...,...,...,...,...,...,...,...
41611,2023-09-30,19,518.905,7.9,0.0,0,"дымка, прохладно"
41612,2023-09-30,20,536.106,7.9,0.0,0,"дымка, прохладно"
41613,2023-09-30,21,524.275,7.5,NaN,NaN,"дымка, прохладно"
41614,2023-09-30,22,503.367,7.5,0.0,0,"дымка, прохладно"


In [7]:
train_ds = transformer.transform(train_ds)

/Users/maxsemakov/Documents/glowbyte/first-competition-1/data_preprocess.py:244: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_ds[column+'_'+str(lag)] = train_ds[column].shift(lag)
/Users/maxsemakov/Documents/glowbyte/first-competition-1/data_preprocess.py:244: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_ds[column+'_'+str(lag)] = train_ds[column].shift(lag)
/Users/maxsemakov/Documents/glowbyte/first-competition-1/data_preprocess.py:244: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result

In [8]:
train_ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41616 entries, 0 to 41615
Columns: 806 entries, date to temp_9
dtypes: datetime64[ns](1), float64(789), int32(5), int64(9), object(2)
memory usage: 255.1+ MB


#### 1.11 Исключение лишних колонок

In [9]:
# Отбираем признаки. Все лишние колонки здесь отбрасываем, кроме 'date', которую уберем позже 

feature_cols = list(train_ds.columns)

# выбрасываем взгляд в прошлое и расшифрованную погоду
drop_list = ['target', 'day_of_year', 'weather_pred', 'weather_fact', 'temp', 
             'target_lag_48', 'target_lag_168']


for name in drop_list:
    feature_cols.remove(name)

# Итоговый список признаков
feature_cols

['date',
 'time',
 'temp_pred',
 'year',
 'month',
 'day_of_week',
 'day',
 'holidays_true',
 'preholidays_true',
 'temp_last_day',
 'target_lag_24',
 'target_lag_72',
 'target_lag_336',
 'VVP',
 'cloudy',
 'rainy',
 'windy',
 'clear',
 'rain_probability',
 'has_rain_probability',
 'P',
 'U',
 'WW',
 'Td',
 'N',
 'S',
 'W',
 'E',
 'last_evening_avg_target_0',
 'last_evening_avg_temp_0',
 'last_evening_avg_target_19',
 'last_evening_avg_temp_19',
 'last_evening_avg_target_22',
 'last_evening_avg_temp_22',
 'time_1',
 'temp_pred_1',
 'year_1',
 'month_1',
 'day_of_week_1',
 'day_1',
 'holidays_true_1',
 'preholidays_true_1',
 'temp_last_day_1',
 'target_lag_24_1',
 'target_lag_72_1',
 'target_lag_336_1',
 'VVP_1',
 'cloudy_1',
 'rainy_1',
 'windy_1',
 'clear_1',
 'rain_probability_1',
 'has_rain_probability_1',
 'P_1',
 'U_1',
 'WW_1',
 'Td_1',
 'N_1',
 'S_1',
 'W_1',
 'E_1',
 'last_evening_avg_target_0_1',
 'last_evening_avg_temp_0_1',
 'last_evening_avg_target_19_1',
 'last_evening_avg

In [10]:
# Формируем набор датасетов для обучения и проверки
features = train_ds[feature_cols]
target = train_ds['target']

# Функция для выделения временных интервалов из таблиц признаков и целей
# на этом этапе отбрасываем колонку 'date'


# для проверки на тестовой выборке будем учиться на всем тренировочном датасете

features_all_train, target_all_train = transformer.features_interval(features, target, '2019-01-01', test_begin)
features_test, target_test = transformer.features_interval(features, target, test_begin, test_end)



FEATURES = 'test_summer_fulltest'



### 4. Проверка метрик на тестовом датасете

#### 4.1 LGBM

In [11]:

feat_lgbm_train = features_all_train
feat_lgbm_test = features_test

feat_lgbm_train.columns, feat_lgbm_test.columns

(Index(['time', 'temp_pred', 'year', 'month', 'day_of_week', 'day',
        'holidays_true', 'preholidays_true', 'temp_last_day', 'target_lag_24',
        ...
        'last_evening_avg_target_19_23', 'last_evening_avg_temp_19_23',
        'last_evening_avg_target_22_23', 'last_evening_avg_temp_22_23',
        'target_1', 'temp_1', 'target_5', 'temp_5', 'target_9', 'temp_9'],
       dtype='object', length=798),
 Index(['time', 'temp_pred', 'year', 'month', 'day_of_week', 'day',
        'holidays_true', 'preholidays_true', 'temp_last_day', 'target_lag_24',
        ...
        'last_evening_avg_target_19_23', 'last_evening_avg_temp_19_23',
        'last_evening_avg_target_22_23', 'last_evening_avg_temp_22_23',
        'target_1', 'temp_1', 'target_5', 'temp_5', 'target_9', 'temp_9'],
       dtype='object', length=798))

In [12]:
feat_lgbm = ['time', 'temp_pred', 'year', 'month', 'day_of_week', 'day', 'holidays_true', 'preholidays_true', 'temp_last_day', 'target_lag_24', 'target_lag_72', 'target_lag_336', 'VVP', 'cloudy', 'rainy', 'windy', 'clear', 'rain_probability', 'has_rain_probability', 'P', 'U', 'WW', 'Td', 'N', 'S', 'W', 'E', 'last_evening_avg_target_0', 'last_evening_avg_temp_0', 'last_evening_avg_target_19', 'last_evening_avg_temp_19', 'last_evening_avg_target_22', 'last_evening_avg_temp_22', 'time_1', 'temp_pred_1', 'year_1', 'month_1', 'day_of_week_1', 'day_1', 'holidays_true_1', 'preholidays_true_1', 'temp_last_day_1', 'target_lag_24_1', 'target_lag_72_1', 'target_lag_336_1', 'VVP_1', 'cloudy_1', 'rainy_1', 'windy_1', 'clear_1', 'rain_probability_1', 'has_rain_probability_1', 'P_1', 'U_1', 'WW_1', 'Td_1', 'N_1', 'S_1', 'W_1', 'E_1', 'last_evening_avg_target_0_1', 'last_evening_avg_temp_0_1', 'last_evening_avg_target_19_1', 'last_evening_avg_temp_19_1', 'last_evening_avg_target_22_1', 'last_evening_avg_temp_22_1', 'time_2', 'temp_pred_2', 'year_2', 'month_2', 'day_of_week_2', 'day_2', 'holidays_true_2', 'preholidays_true_2', 'temp_last_day_2', 'target_lag_24_2', 'target_lag_72_2', 'target_lag_336_2', 'VVP_2', 'cloudy_2', 'rainy_2', 'windy_2', 'clear_2', 'rain_probability_2', 'has_rain_probability_2', 'P_2', 'U_2', 'WW_2', 'Td_2', 'N_2', 'S_2', 'W_2', 'E_2', 'last_evening_avg_target_0_2', 'last_evening_avg_temp_0_2', 'last_evening_avg_target_19_2', 'last_evening_avg_temp_19_2', 'last_evening_avg_target_22_2', 'last_evening_avg_temp_22_2', 'time_3', 'temp_pred_3', 'year_3', 'month_3', 'day_of_week_3', 'day_3', 'holidays_true_3', 'preholidays_true_3', 'temp_last_day_3', 'target_lag_24_3', 'target_lag_72_3', 'target_lag_336_3', 'VVP_3', 'cloudy_3', 'rainy_3', 'windy_3', 'clear_3', 'rain_probability_3', 'has_rain_probability_3', 'P_3', 'U_3', 'WW_3', 'Td_3', 'N_3', 'S_3', 'W_3', 'E_3', 'last_evening_avg_target_0_3', 'last_evening_avg_temp_0_3', 'last_evening_avg_target_19_3', 'last_evening_avg_temp_19_3', 'last_evening_avg_target_22_3', 'last_evening_avg_temp_22_3', 'time_4', 'temp_pred_4', 'year_4', 'month_4', 'day_of_week_4', 'day_4', 'holidays_true_4', 'preholidays_true_4', 'temp_last_day_4', 'target_lag_24_4', 'target_lag_72_4', 'target_lag_336_4', 'VVP_4', 'cloudy_4', 'rainy_4', 'windy_4', 'clear_4', 'rain_probability_4', 'has_rain_probability_4', 'P_4', 'U_4', 'WW_4', 'Td_4', 'N_4', 'S_4', 'W_4', 'E_4', 'last_evening_avg_target_0_4', 'last_evening_avg_temp_0_4', 'last_evening_avg_target_19_4', 'last_evening_avg_temp_19_4', 'last_evening_avg_target_22_4', 'last_evening_avg_temp_22_4', 'time_5', 'temp_pred_5', 'year_5', 'month_5', 'day_of_week_5', 'day_5', 'holidays_true_5', 'preholidays_true_5', 'temp_last_day_5', 'target_lag_24_5', 'target_lag_72_5', 'target_lag_336_5', 'VVP_5', 'cloudy_5', 'rainy_5', 'windy_5', 'clear_5', 'rain_probability_5', 'has_rain_probability_5', 'P_5', 'U_5', 'WW_5', 'Td_5', 'N_5', 'S_5', 'W_5', 'E_5', 'last_evening_avg_target_0_5', 'last_evening_avg_temp_0_5', 'last_evening_avg_target_19_5', 'last_evening_avg_temp_19_5', 'last_evening_avg_target_22_5', 'last_evening_avg_temp_22_5', 'time_6', 'temp_pred_6', 'year_6', 'month_6', 'day_of_week_6', 'day_6', 'holidays_true_6', 'preholidays_true_6', 'temp_last_day_6', 'target_lag_24_6', 'target_lag_72_6', 'target_lag_336_6', 'VVP_6', 'cloudy_6', 'rainy_6', 'windy_6', 'clear_6', 'rain_probability_6', 'has_rain_probability_6', 'P_6', 'U_6', 'WW_6', 'Td_6', 'N_6', 'S_6', 'W_6', 'E_6', 'last_evening_avg_target_0_6', 'last_evening_avg_temp_0_6', 'last_evening_avg_target_19_6', 'last_evening_avg_temp_19_6', 'last_evening_avg_target_22_6', 'last_evening_avg_temp_22_6', 'time_7', 'temp_pred_7', 'year_7', 'month_7', 'day_of_week_7', 'day_7', 'holidays_true_7', 'preholidays_true_7', 'temp_last_day_7', 'target_lag_24_7', 'target_lag_72_7', 'target_lag_336_7', 'VVP_7', 'cloudy_7', 'rainy_7', 'windy_7', 'clear_7', 'rain_probability_7', 'has_rain_probability_7', 'P_7', 'U_7', 'WW_7', 'Td_7', 'N_7', 'S_7', 'W_7', 'E_7', 'last_evening_avg_target_0_7', 'last_evening_avg_temp_0_7', 'last_evening_avg_target_19_7', 'last_evening_avg_temp_19_7', 'last_evening_avg_target_22_7', 'last_evening_avg_temp_22_7', 'time_8', 'temp_pred_8', 'year_8', 'month_8', 'day_of_week_8', 'day_8', 'holidays_true_8', 'preholidays_true_8', 'temp_last_day_8', 'target_lag_24_8', 'target_lag_72_8', 'target_lag_336_8', 'VVP_8', 'cloudy_8', 'rainy_8', 'windy_8', 'clear_8', 'rain_probability_8', 'has_rain_probability_8', 'P_8', 'U_8', 'WW_8', 'Td_8', 'N_8', 'S_8', 'W_8', 'E_8', 'last_evening_avg_target_0_8', 'last_evening_avg_temp_0_8', 'last_evening_avg_target_19_8', 'last_evening_avg_temp_19_8', 'last_evening_avg_target_22_8', 'last_evening_avg_temp_22_8', 'time_9', 'temp_pred_9', 'year_9', 'month_9', 'day_of_week_9', 'day_9', 'holidays_true_9', 'preholidays_true_9', 'temp_last_day_9', 'target_lag_24_9', 'target_lag_72_9', 'target_lag_336_9', 'VVP_9', 'cloudy_9', 'rainy_9', 'windy_9', 'clear_9', 'rain_probability_9', 'has_rain_probability_9', 'P_9', 'U_9', 'WW_9', 'Td_9', 'N_9', 'S_9', 'W_9', 'E_9', 'last_evening_avg_target_0_9', 'last_evening_avg_temp_0_9', 'last_evening_avg_target_19_9', 'last_evening_avg_temp_19_9', 'last_evening_avg_target_22_9', 'last_evening_avg_temp_22_9', 'time_10', 'temp_pred_10', 'year_10', 'month_10', 'day_of_week_10', 'day_10', 'holidays_true_10', 'preholidays_true_10', 'temp_last_day_10', 'target_lag_24_10', 'target_lag_72_10', 'target_lag_336_10', 'VVP_10', 'cloudy_10', 'rainy_10', 'windy_10', 'clear_10', 'rain_probability_10', 'has_rain_probability_10', 'P_10', 'U_10', 'WW_10', 'Td_10', 'N_10', 'S_10', 'W_10', 'E_10', 'last_evening_avg_target_0_10', 'last_evening_avg_temp_0_10', 'last_evening_avg_target_19_10', 'last_evening_avg_temp_19_10', 'last_evening_avg_target_22_10', 'last_evening_avg_temp_22_10', 'time_11', 'temp_pred_11', 'year_11', 'month_11', 'day_of_week_11', 'day_11', 'holidays_true_11', 'preholidays_true_11', 'temp_last_day_11', 'target_lag_24_11', 'target_lag_72_11', 'target_lag_336_11', 'VVP_11', 'cloudy_11', 'rainy_11', 'windy_11', 'clear_11', 'rain_probability_11', 'has_rain_probability_11', 'P_11', 'U_11', 'WW_11', 'Td_11', 'N_11', 'S_11', 'W_11', 'E_11', 'last_evening_avg_target_0_11', 'last_evening_avg_temp_0_11', 'last_evening_avg_target_19_11', 'last_evening_avg_temp_19_11', 'last_evening_avg_target_22_11', 'last_evening_avg_temp_22_11', 'time_12', 'temp_pred_12', 'year_12', 'month_12', 'day_of_week_12', 'day_12', 'holidays_true_12', 'preholidays_true_12', 'temp_last_day_12', 'target_lag_24_12', 'target_lag_72_12', 'target_lag_336_12', 'VVP_12', 'cloudy_12', 'rainy_12', 'windy_12', 'clear_12', 'rain_probability_12', 'has_rain_probability_12', 'P_12', 'U_12', 'WW_12', 'Td_12', 'N_12', 'S_12', 'W_12', 'E_12', 'last_evening_avg_target_0_12', 'last_evening_avg_temp_0_12', 'last_evening_avg_target_19_12', 'last_evening_avg_temp_19_12', 'last_evening_avg_target_22_12', 'last_evening_avg_temp_22_12', 'time_13', 'temp_pred_13', 'year_13', 'month_13', 'day_of_week_13', 'day_13', 'holidays_true_13', 'preholidays_true_13', 'temp_last_day_13', 'target_lag_24_13', 'target_lag_72_13', 'target_lag_336_13', 'VVP_13', 'cloudy_13', 'rainy_13', 'windy_13', 'clear_13', 'rain_probability_13', 'has_rain_probability_13', 'P_13', 'U_13', 'WW_13', 'Td_13', 'N_13', 'S_13', 'W_13', 'E_13', 'last_evening_avg_target_0_13', 'last_evening_avg_temp_0_13', 'last_evening_avg_target_19_13', 'last_evening_avg_temp_19_13', 'last_evening_avg_target_22_13', 'last_evening_avg_temp_22_13', 'time_14', 'temp_pred_14', 'year_14', 'month_14', 'day_of_week_14', 'day_14', 'holidays_true_14', 'preholidays_true_14', 'temp_last_day_14', 'target_lag_24_14', 'target_lag_72_14', 'target_lag_336_14', 'VVP_14', 'cloudy_14', 'rainy_14', 'windy_14', 'clear_14', 'rain_probability_14', 'has_rain_probability_14', 'P_14', 'U_14', 'WW_14', 'Td_14', 'N_14', 'S_14', 'W_14', 'E_14', 'last_evening_avg_target_0_14', 'last_evening_avg_temp_0_14', 'last_evening_avg_target_19_14', 'last_evening_avg_temp_19_14', 'last_evening_avg_target_22_14', 'last_evening_avg_temp_22_14', 'time_15', 'temp_pred_15', 'year_15', 'month_15', 'day_of_week_15', 'day_15', 'holidays_true_15', 'preholidays_true_15', 'temp_last_day_15', 'target_lag_24_15', 'target_lag_72_15', 'target_lag_336_15', 'VVP_15', 'cloudy_15', 'rainy_15', 'windy_15', 'clear_15', 'rain_probability_15', 'has_rain_probability_15', 'P_15', 'U_15', 'WW_15', 'Td_15', 'N_15', 'S_15', 'W_15', 'E_15', 'last_evening_avg_target_0_15', 'last_evening_avg_temp_0_15', 'last_evening_avg_target_19_15', 'last_evening_avg_temp_19_15', 'last_evening_avg_target_22_15', 'last_evening_avg_temp_22_15', 'time_16', 'temp_pred_16', 'year_16', 'month_16', 'day_of_week_16', 'day_16', 'holidays_true_16', 'preholidays_true_16', 'temp_last_day_16', 'target_lag_24_16', 'target_lag_72_16', 'target_lag_336_16', 'VVP_16', 'cloudy_16', 'rainy_16', 'windy_16', 'clear_16', 'rain_probability_16', 'has_rain_probability_16', 'P_16', 'U_16', 'WW_16', 'Td_16', 'N_16', 'S_16', 'W_16', 'E_16', 'last_evening_avg_target_0_16', 'last_evening_avg_temp_0_16', 'last_evening_avg_target_19_16', 'last_evening_avg_temp_19_16', 'last_evening_avg_target_22_16', 'last_evening_avg_temp_22_16', 'time_17', 'temp_pred_17', 'year_17', 'month_17', 'day_of_week_17', 'day_17', 'holidays_true_17', 'preholidays_true_17', 'temp_last_day_17', 'target_lag_24_17', 'target_lag_72_17', 'target_lag_336_17', 'VVP_17', 'cloudy_17', 'rainy_17', 'windy_17', 'clear_17', 'rain_probability_17', 'has_rain_probability_17', 'P_17', 'U_17', 'WW_17', 'Td_17', 'N_17', 'S_17', 'W_17', 'E_17', 'last_evening_avg_target_0_17', 'last_evening_avg_temp_0_17', 'last_evening_avg_target_19_17', 'last_evening_avg_temp_19_17', 'last_evening_avg_target_22_17', 'last_evening_avg_temp_22_17', 'time_18', 'temp_pred_18', 'year_18', 'month_18', 'day_of_week_18', 'day_18', 'holidays_true_18', 'preholidays_true_18', 'temp_last_day_18', 'target_lag_24_18', 'target_lag_72_18', 'target_lag_336_18', 'VVP_18', 'cloudy_18', 'rainy_18', 'windy_18', 'clear_18', 'rain_probability_18', 'has_rain_probability_18', 'P_18', 'U_18', 'WW_18', 'Td_18', 'N_18', 'S_18', 'W_18', 'E_18', 'last_evening_avg_target_0_18', 'last_evening_avg_temp_0_18', 'last_evening_avg_target_19_18', 'last_evening_avg_temp_19_18', 'last_evening_avg_target_22_18', 'last_evening_avg_temp_22_18', 'time_19', 'temp_pred_19', 'year_19', 'month_19', 'day_of_week_19', 'day_19', 'holidays_true_19', 'preholidays_true_19', 'temp_last_day_19', 'target_lag_24_19', 'target_lag_72_19', 'target_lag_336_19', 'VVP_19', 'cloudy_19', 'rainy_19', 'windy_19', 'clear_19', 'rain_probability_19', 'has_rain_probability_19', 'P_19', 'U_19', 'WW_19', 'Td_19', 'N_19', 'S_19', 'W_19', 'E_19', 'last_evening_avg_target_0_19', 'last_evening_avg_temp_0_19', 'last_evening_avg_target_19_19', 'last_evening_avg_temp_19_19', 'last_evening_avg_target_22_19', 'last_evening_avg_temp_22_19', 'time_20', 'temp_pred_20', 'year_20', 'month_20', 'day_of_week_20', 'day_20', 'holidays_true_20', 'preholidays_true_20', 'temp_last_day_20', 'target_lag_24_20', 'target_lag_72_20', 'target_lag_336_20', 'VVP_20', 'cloudy_20', 'rainy_20', 'windy_20', 'clear_20', 'rain_probability_20', 'has_rain_probability_20', 'P_20', 'U_20', 'WW_20', 'Td_20', 'N_20', 'S_20', 'W_20', 'E_20', 'last_evening_avg_target_0_20', 'last_evening_avg_temp_0_20', 'last_evening_avg_target_19_20', 'last_evening_avg_temp_19_20', 'last_evening_avg_target_22_20', 'last_evening_avg_temp_22_20', 'time_21', 'temp_pred_21', 'year_21', 'month_21', 'day_of_week_21', 'day_21', 'holidays_true_21', 'preholidays_true_21', 'temp_last_day_21', 'target_lag_24_21', 'target_lag_72_21', 'target_lag_336_21', 'VVP_21', 'cloudy_21', 'rainy_21', 'windy_21', 'clear_21', 'rain_probability_21', 'has_rain_probability_21', 'P_21', 'U_21', 'WW_21', 'Td_21', 'N_21', 'S_21', 'W_21', 'E_21', 'last_evening_avg_target_0_21', 'last_evening_avg_temp_0_21', 'last_evening_avg_target_19_21', 'last_evening_avg_temp_19_21', 'last_evening_avg_target_22_21', 'last_evening_avg_temp_22_21', 'time_22', 'temp_pred_22', 'year_22', 'month_22', 'day_of_week_22', 'day_22', 'holidays_true_22', 'preholidays_true_22', 'temp_last_day_22', 'target_lag_24_22', 'target_lag_72_22', 'target_lag_336_22', 'VVP_22', 'cloudy_22', 'rainy_22', 'windy_22', 'clear_22', 'rain_probability_22', 'has_rain_probability_22', 'P_22', 'U_22', 'WW_22', 'Td_22', 'N_22', 'S_22', 'W_22', 'E_22', 'last_evening_avg_target_0_22', 'last_evening_avg_temp_0_22', 'last_evening_avg_target_19_22', 'last_evening_avg_temp_19_22', 'last_evening_avg_target_22_22', 'last_evening_avg_temp_22_22', 'time_23', 'temp_pred_23', 'year_23', 'month_23', 'day_of_week_23', 'day_23', 'holidays_true_23', 'preholidays_true_23', 'temp_last_day_23', 'target_lag_24_23', 'target_lag_72_23', 'target_lag_336_23', 'VVP_23', 'cloudy_23', 'rainy_23', 'windy_23', 'clear_23', 'rain_probability_23', 'has_rain_probability_23', 'P_23', 'U_23', 'WW_23', 'Td_23', 'N_23', 'S_23', 'W_23', 'E_23', 'last_evening_avg_target_0_23', 'last_evening_avg_temp_0_23', 'last_evening_avg_target_19_23', 'last_evening_avg_temp_19_23', 'last_evening_avg_target_22_23', 'last_evening_avg_temp_22_23', 'target_1', 'temp_1', 'target_5', 'temp_5', 'target_9', 'temp_9']


In [13]:
feat_lgbm_train_2 = features_all_train[feat_lgbm]
feat_lgbm_test_2 = features_test[feat_lgbm]

feat_lgbm_train_2.columns, feat_lgbm_test_2.columns

(Index(['time', 'temp_pred', 'year', 'month', 'day_of_week', 'day',
        'holidays_true', 'preholidays_true', 'temp_last_day', 'target_lag_24',
        ...
        'last_evening_avg_target_19_23', 'last_evening_avg_temp_19_23',
        'last_evening_avg_target_22_23', 'last_evening_avg_temp_22_23',
        'target_1', 'temp_1', 'target_5', 'temp_5', 'target_9', 'temp_9'],
       dtype='object', length=798),
 Index(['time', 'temp_pred', 'year', 'month', 'day_of_week', 'day',
        'holidays_true', 'preholidays_true', 'temp_last_day', 'target_lag_24',
        ...
        'last_evening_avg_target_19_23', 'last_evening_avg_temp_19_23',
        'last_evening_avg_target_22_23', 'last_evening_avg_temp_22_23',
        'target_1', 'temp_1', 'target_5', 'temp_5', 'target_9', 'temp_9'],
       dtype='object', length=798))

In [14]:
cols = feat_lgbm_test_2.columns

In [15]:
print((cols == feat_lgbm))

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  T

In [16]:
for i, (a, b) in enumerate(zip(cols, feat_lgbm)):
    if a != b:
        print(f"Элементы на позиции {i} не совпадают: {a} != {b}")

In [17]:
print(set(feature_cols).issubset(feat_lgbm))

False


In [18]:
set_diff1 = set(feat_lgbm) - set(feat_lgbm)
print('Элементы в list1, которых нет в list2:', set_diff1)

Элементы в list1, которых нет в list2: set()


In [24]:
# Проверка метрики лучшей модели на тестовом датасете
# Здесь обучаем на всем тренировочном датасете

lgbm_model_all_train = lgb.Booster(model_file='models/lgb_model_FULL_ds_no_aug.txt')

l_predict_train = lgbm_model_all_train.predict(feat_lgbm_train)
l_predict_test = lgbm_model_all_train.predict(feat_lgbm_test)

mae_train, mape_train, r2_train = metrics_hour(target_all_train, l_predict_train)
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_test, l_predict_test)

results = pd.DataFrame([[f'тренировочная LGBM {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая LGBM {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))

/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_

In [25]:
results

,Выборка,MAE,MAPE,R2
0,тренировочная LGBM test_summer_fulltest,3.318779,0.007010,0.996668
1,тестовая LGBM test_summer_fulltest,11.590861,0.026719,0.930641


#### 4.2 XGBoost

In [26]:
features_all_train.columns

Index(['time', 'temp_pred', 'year', 'month', 'day_of_week', 'day',
       'holidays_true', 'preholidays_true', 'temp_last_day', 'target_lag_24',
       ...
       'last_evening_avg_target_19_23', 'last_evening_avg_temp_19_23',
       'last_evening_avg_target_22_23', 'last_evening_avg_temp_22_23',
       'target_1', 'temp_1', 'target_5', 'temp_5', 'target_9', 'temp_9'],
      dtype='object', length=798)

In [27]:
drop_list = ['preholidays_true']
n_values = range(1, 24)
preholidays = ['preholidays_true_{}'.format(n) for n in n_values]


drop_list = drop_list + preholidays #+ has_rain + W_wind + E_wind

feat_xgb_test = features_test.drop(columns=drop_list)
feat_xgb_train = features_all_train.drop(columns=drop_list)
feat_xgb_test.columns

Index(['time', 'temp_pred', 'year', 'month', 'day_of_week', 'day',
       'holidays_true', 'temp_last_day', 'target_lag_24', 'target_lag_72',
       ...
       'last_evening_avg_target_19_23', 'last_evening_avg_temp_19_23',
       'last_evening_avg_target_22_23', 'last_evening_avg_temp_22_23',
       'target_1', 'temp_1', 'target_5', 'temp_5', 'target_9', 'temp_9'],
      dtype='object', length=774)

In [29]:
xgb_feat = ["time","temp_pred","year","month","day_of_week","day","holidays_true","temp_last_day","target_lag_24","target_lag_72","target_lag_336","VVP","cloudy","rainy","windy","clear","rain_probability","has_rain_probability","P","U","WW","Td","N","S","W","E","last_evening_avg_target_0","last_evening_avg_temp_0","last_evening_avg_target_19","last_evening_avg_temp_19","last_evening_avg_target_22","last_evening_avg_temp_22","time_1","temp_pred_1","year_1","month_1","day_of_week_1","day_1","holidays_true_1","temp_last_day_1","target_lag_24_1","target_lag_72_1","target_lag_336_1","VVP_1","cloudy_1","rainy_1","windy_1","clear_1","rain_probability_1","has_rain_probability_1","P_1","U_1","WW_1","Td_1","N_1","S_1","W_1","E_1","last_evening_avg_target_0_1","last_evening_avg_temp_0_1","last_evening_avg_target_19_1","last_evening_avg_temp_19_1","last_evening_avg_target_22_1","last_evening_avg_temp_22_1","time_2","temp_pred_2","year_2","month_2","day_of_week_2","day_2","holidays_true_2","temp_last_day_2","target_lag_24_2","target_lag_72_2","target_lag_336_2","VVP_2","cloudy_2","rainy_2","windy_2","clear_2","rain_probability_2","has_rain_probability_2","P_2","U_2","WW_2","Td_2","N_2","S_2","W_2","E_2","last_evening_avg_target_0_2","last_evening_avg_temp_0_2","last_evening_avg_target_19_2","last_evening_avg_temp_19_2","last_evening_avg_target_22_2","last_evening_avg_temp_22_2","time_3","temp_pred_3","year_3","month_3","day_of_week_3","day_3","holidays_true_3","temp_last_day_3","target_lag_24_3","target_lag_72_3","target_lag_336_3","VVP_3","cloudy_3","rainy_3","windy_3","clear_3","rain_probability_3","has_rain_probability_3","P_3","U_3","WW_3","Td_3","N_3","S_3","W_3","E_3","last_evening_avg_target_0_3","last_evening_avg_temp_0_3","last_evening_avg_target_19_3","last_evening_avg_temp_19_3","last_evening_avg_target_22_3","last_evening_avg_temp_22_3","time_4","temp_pred_4","year_4","month_4","day_of_week_4","day_4","holidays_true_4","temp_last_day_4","target_lag_24_4","target_lag_72_4","target_lag_336_4","VVP_4","cloudy_4","rainy_4","windy_4","clear_4","rain_probability_4","has_rain_probability_4","P_4","U_4","WW_4","Td_4","N_4","S_4","W_4","E_4","last_evening_avg_target_0_4","last_evening_avg_temp_0_4","last_evening_avg_target_19_4","last_evening_avg_temp_19_4","last_evening_avg_target_22_4","last_evening_avg_temp_22_4","time_5","temp_pred_5","year_5","month_5","day_of_week_5","day_5","holidays_true_5","temp_last_day_5","target_lag_24_5","target_lag_72_5","target_lag_336_5","VVP_5","cloudy_5","rainy_5","windy_5","clear_5","rain_probability_5","has_rain_probability_5","P_5","U_5","WW_5","Td_5","N_5","S_5","W_5","E_5","last_evening_avg_target_0_5","last_evening_avg_temp_0_5","last_evening_avg_target_19_5","last_evening_avg_temp_19_5","last_evening_avg_target_22_5","last_evening_avg_temp_22_5","time_6","temp_pred_6","year_6","month_6","day_of_week_6","day_6","holidays_true_6","temp_last_day_6","target_lag_24_6","target_lag_72_6","target_lag_336_6","VVP_6","cloudy_6","rainy_6","windy_6","clear_6","rain_probability_6","has_rain_probability_6","P_6","U_6","WW_6","Td_6","N_6","S_6","W_6","E_6","last_evening_avg_target_0_6","last_evening_avg_temp_0_6","last_evening_avg_target_19_6","last_evening_avg_temp_19_6","last_evening_avg_target_22_6","last_evening_avg_temp_22_6","time_7","temp_pred_7","year_7","month_7","day_of_week_7","day_7","holidays_true_7","temp_last_day_7","target_lag_24_7","target_lag_72_7","target_lag_336_7","VVP_7","cloudy_7","rainy_7","windy_7","clear_7","rain_probability_7","has_rain_probability_7","P_7","U_7","WW_7","Td_7","N_7","S_7","W_7","E_7","last_evening_avg_target_0_7","last_evening_avg_temp_0_7","last_evening_avg_target_19_7","last_evening_avg_temp_19_7","last_evening_avg_target_22_7","last_evening_avg_temp_22_7","time_8","temp_pred_8","year_8","month_8","day_of_week_8","day_8","holidays_true_8","temp_last_day_8","target_lag_24_8","target_lag_72_8","target_lag_336_8","VVP_8","cloudy_8","rainy_8","windy_8","clear_8","rain_probability_8","has_rain_probability_8","P_8","U_8","WW_8","Td_8","N_8","S_8","W_8","E_8","last_evening_avg_target_0_8","last_evening_avg_temp_0_8","last_evening_avg_target_19_8","last_evening_avg_temp_19_8","last_evening_avg_target_22_8","last_evening_avg_temp_22_8","time_9","temp_pred_9","year_9","month_9","day_of_week_9","day_9","holidays_true_9","temp_last_day_9","target_lag_24_9","target_lag_72_9","target_lag_336_9","VVP_9","cloudy_9","rainy_9","windy_9","clear_9","rain_probability_9","has_rain_probability_9","P_9","U_9","WW_9","Td_9","N_9","S_9","W_9","E_9","last_evening_avg_target_0_9","last_evening_avg_temp_0_9","last_evening_avg_target_19_9","last_evening_avg_temp_19_9","last_evening_avg_target_22_9","last_evening_avg_temp_22_9","time_10","temp_pred_10","year_10","month_10","day_of_week_10","day_10","holidays_true_10","temp_last_day_10","target_lag_24_10","target_lag_72_10","target_lag_336_10","VVP_10","cloudy_10","rainy_10","windy_10","clear_10","rain_probability_10","has_rain_probability_10","P_10","U_10","WW_10","Td_10","N_10","S_10","W_10","E_10","last_evening_avg_target_0_10","last_evening_avg_temp_0_10","last_evening_avg_target_19_10","last_evening_avg_temp_19_10","last_evening_avg_target_22_10","last_evening_avg_temp_22_10","time_11","temp_pred_11","year_11","month_11","day_of_week_11","day_11","holidays_true_11","temp_last_day_11","target_lag_24_11","target_lag_72_11","target_lag_336_11","VVP_11","cloudy_11","rainy_11","windy_11","clear_11","rain_probability_11","has_rain_probability_11","P_11","U_11","WW_11","Td_11","N_11","S_11","W_11","E_11","last_evening_avg_target_0_11","last_evening_avg_temp_0_11","last_evening_avg_target_19_11","last_evening_avg_temp_19_11","last_evening_avg_target_22_11","last_evening_avg_temp_22_11","time_12","temp_pred_12","year_12","month_12","day_of_week_12","day_12","holidays_true_12","temp_last_day_12","target_lag_24_12","target_lag_72_12","target_lag_336_12","VVP_12","cloudy_12","rainy_12","windy_12","clear_12","rain_probability_12","has_rain_probability_12","P_12","U_12","WW_12","Td_12","N_12","S_12","W_12","E_12","last_evening_avg_target_0_12","last_evening_avg_temp_0_12","last_evening_avg_target_19_12","last_evening_avg_temp_19_12","last_evening_avg_target_22_12","last_evening_avg_temp_22_12","time_13","temp_pred_13","year_13","month_13","day_of_week_13","day_13","holidays_true_13","temp_last_day_13","target_lag_24_13","target_lag_72_13","target_lag_336_13","VVP_13","cloudy_13","rainy_13","windy_13","clear_13","rain_probability_13","has_rain_probability_13","P_13","U_13","WW_13","Td_13","N_13","S_13","W_13","E_13","last_evening_avg_target_0_13","last_evening_avg_temp_0_13","last_evening_avg_target_19_13","last_evening_avg_temp_19_13","last_evening_avg_target_22_13","last_evening_avg_temp_22_13","time_14","temp_pred_14","year_14","month_14","day_of_week_14","day_14","holidays_true_14","temp_last_day_14","target_lag_24_14","target_lag_72_14","target_lag_336_14","VVP_14","cloudy_14","rainy_14","windy_14","clear_14","rain_probability_14","has_rain_probability_14","P_14","U_14","WW_14","Td_14","N_14","S_14","W_14","E_14","last_evening_avg_target_0_14","last_evening_avg_temp_0_14","last_evening_avg_target_19_14","last_evening_avg_temp_19_14","last_evening_avg_target_22_14","last_evening_avg_temp_22_14","time_15","temp_pred_15","year_15","month_15","day_of_week_15","day_15","holidays_true_15","temp_last_day_15","target_lag_24_15","target_lag_72_15","target_lag_336_15","VVP_15","cloudy_15","rainy_15","windy_15","clear_15","rain_probability_15","has_rain_probability_15","P_15","U_15","WW_15","Td_15","N_15","S_15","W_15","E_15","last_evening_avg_target_0_15","last_evening_avg_temp_0_15","last_evening_avg_target_19_15","last_evening_avg_temp_19_15","last_evening_avg_target_22_15","last_evening_avg_temp_22_15","time_16","temp_pred_16","year_16","month_16","day_of_week_16","day_16","holidays_true_16","temp_last_day_16","target_lag_24_16","target_lag_72_16","target_lag_336_16","VVP_16","cloudy_16","rainy_16","windy_16","clear_16","rain_probability_16","has_rain_probability_16","P_16","U_16","WW_16","Td_16","N_16","S_16","W_16","E_16","last_evening_avg_target_0_16","last_evening_avg_temp_0_16","last_evening_avg_target_19_16","last_evening_avg_temp_19_16","last_evening_avg_target_22_16","last_evening_avg_temp_22_16","time_17","temp_pred_17","year_17","month_17","day_of_week_17","day_17","holidays_true_17","temp_last_day_17","target_lag_24_17","target_lag_72_17","target_lag_336_17","VVP_17","cloudy_17","rainy_17","windy_17","clear_17","rain_probability_17","has_rain_probability_17","P_17","U_17","WW_17","Td_17","N_17","S_17","W_17","E_17","last_evening_avg_target_0_17","last_evening_avg_temp_0_17","last_evening_avg_target_19_17","last_evening_avg_temp_19_17","last_evening_avg_target_22_17","last_evening_avg_temp_22_17","time_18","temp_pred_18","year_18","month_18","day_of_week_18","day_18","holidays_true_18","temp_last_day_18","target_lag_24_18","target_lag_72_18","target_lag_336_18","VVP_18","cloudy_18","rainy_18","windy_18","clear_18","rain_probability_18","has_rain_probability_18","P_18","U_18","WW_18","Td_18","N_18","S_18","W_18","E_18","last_evening_avg_target_0_18","last_evening_avg_temp_0_18","last_evening_avg_target_19_18","last_evening_avg_temp_19_18","last_evening_avg_target_22_18","last_evening_avg_temp_22_18","time_19","temp_pred_19","year_19","month_19","day_of_week_19","day_19","holidays_true_19","temp_last_day_19","target_lag_24_19","target_lag_72_19","target_lag_336_19","VVP_19","cloudy_19","rainy_19","windy_19","clear_19","rain_probability_19","has_rain_probability_19","P_19","U_19","WW_19","Td_19","N_19","S_19","W_19","E_19","last_evening_avg_target_0_19","last_evening_avg_temp_0_19","last_evening_avg_target_19_19","last_evening_avg_temp_19_19","last_evening_avg_target_22_19","last_evening_avg_temp_22_19","time_20","temp_pred_20","year_20","month_20","day_of_week_20","day_20","holidays_true_20","temp_last_day_20","target_lag_24_20","target_lag_72_20","target_lag_336_20","VVP_20","cloudy_20","rainy_20","windy_20","clear_20","rain_probability_20","has_rain_probability_20","P_20","U_20","WW_20","Td_20","N_20","S_20","W_20","E_20","last_evening_avg_target_0_20","last_evening_avg_temp_0_20","last_evening_avg_target_19_20","last_evening_avg_temp_19_20","last_evening_avg_target_22_20","last_evening_avg_temp_22_20","time_21","temp_pred_21","year_21","month_21","day_of_week_21","day_21","holidays_true_21","temp_last_day_21","target_lag_24_21","target_lag_72_21","target_lag_336_21","VVP_21","cloudy_21","rainy_21","windy_21","clear_21","rain_probability_21","has_rain_probability_21","P_21","U_21","WW_21","Td_21","N_21","S_21","W_21","E_21","last_evening_avg_target_0_21","last_evening_avg_temp_0_21","last_evening_avg_target_19_21","last_evening_avg_temp_19_21","last_evening_avg_target_22_21","last_evening_avg_temp_22_21","time_22","temp_pred_22","year_22","month_22","day_of_week_22","day_22","holidays_true_22","temp_last_day_22","target_lag_24_22","target_lag_72_22","target_lag_336_22","VVP_22","cloudy_22","rainy_22","windy_22","clear_22","rain_probability_22","has_rain_probability_22","P_22","U_22","WW_22","Td_22","N_22","S_22","W_22","E_22","last_evening_avg_target_0_22","last_evening_avg_temp_0_22","last_evening_avg_target_19_22","last_evening_avg_temp_19_22","last_evening_avg_target_22_22","last_evening_avg_temp_22_22","time_23","temp_pred_23","year_23","month_23","day_of_week_23","day_23","holidays_true_23","temp_last_day_23","target_lag_24_23","target_lag_72_23","target_lag_336_23","VVP_23","cloudy_23","rainy_23","windy_23","clear_23","rain_probability_23","has_rain_probability_23","P_23","U_23","WW_23","Td_23","N_23","S_23","W_23","E_23","last_evening_avg_target_0_23","last_evening_avg_temp_0_23","last_evening_avg_target_19_23","last_evening_avg_temp_19_23","last_evening_avg_target_22_23","last_evening_avg_temp_22_23","target_1","temp_1","target_5","temp_5","target_9","temp_9"]

In [30]:
for i, (a, b) in enumerate(zip(feat_xgb_test.columns, xgb_feat)):
    if a != b:
        print(f"Элементы на позиции {i} не совпадают: {a} != {b}")

In [31]:
# Проверка метрики лучшей модели на тестовом датасете
# Здесь обучаем на всем тренировочном датасете

xgb_model_all_train = XGBRegressor()

# загружаем модель из файла
xgb_model_all_train.load_model('models/xgb_model_FULL_ds_no_aug.json')



In [32]:
xgb_predict_test = xgb_model_all_train.predict(feat_xgb_test)

In [ ]:

xgb_predict_train = xgb_model_all_train.predict(feat_xgb_train)

In [ ]:





mae_train, mape_train, r2_train = metrics_hour(target_all_train, xgb_predict_train )
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_test, xgb_predict_test )

results = pd.concat([results,
pd.DataFrame([[f'тренировочная XGB {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая XGB {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
 ])


/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/anaconda3/envs/p311/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_

In [ ]:
results

,Выборка,MAE,MAPE,R2
0,тренировочная LGBM test_summer_fulltest,5.033212,0.010449,0.994069
1,тестовая LGBM test_summer_fulltest,11.984023,0.027576,0.927352
0,тренировочная XGB test_summer_fulltest,3.959548,0.008391,0.997269
1,тестовая XGB test_summer_fulltest,12.005265,0.027803,0.927532


### 5. Ensemble

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import ElasticNet
from lightgbm import LGBMRegressor
from sklearn.ensemble import StackingRegressor

import optuna
from optuna.samplers import TPESampler

In [ ]:
results_ensemble = pd.DataFrame(columns=('Выборка', 'MAE', 'MAPE', 'R2'))

### Simple Ensemble

In [ ]:

predict = (xgb_predict_test + l_predict_test)/2



In [ ]:

predict_result = pd.DataFrame( )

datetimes = train_ds.loc[features_test.index, 'date'] + pd.to_timedelta(train_ds.loc[features_test.index, 'time'], 'H')
#predict_result['datetime'] = pd.DataFrame(datetimes).reset_index()
predict_result['datetime'] = datetimes.values
#predict_result = predict_result.groupby(by='date').sum().reset_index()
predict_result['predict'] = predict

predict_result.reset_index(drop=True)

,datetime,predict
0,2023-08-01 00:00:00,374.452221
1,2023-08-01 01:00:00,340.498075
2,2023-08-01 02:00:00,320.504567
3,2023-08-01 03:00:00,306.396488
4,2023-08-01 04:00:00,300.349185
...,...,...
1459,2023-09-30 19:00:00,510.182327
1460,2023-09-30 20:00:00,529.854040
1461,2023-09-30 21:00:00,518.370388
1462,2023-09-30 22:00:00,497.100690


In [ ]:
predict_result

In [ ]:
mae_train, mape_train, r2_train = metrics_hour(target_all_train, predict_simple_ensemble_train)
mae_open_test, mape_open_test, r2_open_test = metrics_hour(target_test, predict_simple_ensemble_test)

results_ensemble = pd.concat([results_ensemble,
pd.DataFrame([[f'тренировочная simple_ensemble  {FEATURES}', mae_train, mape_train, r2_train], [f'тестовая simple_ensemble {FEATURES}', mae_open_test, mape_open_test, r2_open_test]], 
             columns=('Выборка', 'MAE', 'MAPE', 'R2'))
 ])

In [ ]:
display(results_ensemble)
results

In [ ]:
mae_day(target_open_test, predict_simple_ensemble_test)

import os

# определите путь к папке, которую вы хотите создать
folder_path = "models"

# проверьте, существует ли уже папка
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

lgbm_model_all_train.booster_.save_model('models/lgb_model.txt')
xgb_model.save_model('models/xgb_model.json')

In [ ]:
results.to_csv(f'models/results_LGBM_XGBoost_simple_ensemble_{FEATURES}')